# 移動時間と準備時間の予測

<div style="text-align: right;">
2021/07/13
</div>


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import PySimpleGUI as sg
from urllib import request
import requests
import io
import math
import os
import datetime
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import ShuffleSplit

In [5]:
data1=pd.read_csv("kisyou.csv")
data2=pd.read_csv("mainkisyou.csv")
data3=pd.read_csv("testdata.csv")

In [6]:
#平均気温データ学習フレーム
df=pd.DataFrame()
df["datetime"]=pd.to_datetime(data2.loc[366:,"年月日"])
df["平均気温"]=data2["平均気温"]
df=df.set_index("datetime",drop=True)
#平均風速データ学習フレーム
df2=pd.DataFrame()
df2["datetime"]=pd.to_datetime(data2.loc[366:,"年月日"])
df2["降水量の合計"]=data2["降水量の合計"]
df2=df2.set_index("datetime",drop=True)
#平均雲量データ学習フレーム
df3=pd.DataFrame()
df3["datetime"]=pd.to_datetime(data2.loc[366:,"年月日"])
df3["平均雲量"]=data2["平均雲量"]
df3=df3.set_index("datetime",drop=True)
#降水量データ学習フレーム
df4=pd.DataFrame()
df4["datetime"]=pd.to_datetime(data2.loc[366:,"年月日"])
df4["平均風速"]=data2["平均風速"]
df4=df4.set_index("datetime",drop=True)


In [7]:
def dataset1(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=data2.平均気温.loc[366+j-i]
        set[str("平均気温 -")+str(i)]=tmp[0][i]
    return set
def dataset2(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=data2.降水量の合計.loc[366+j-i]
        set[str("降水量の合計 -")+str(i)]=tmp[0][i]
    return set
def dataset3(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=data2.平均雲量.loc[366+j-i]
        set[str("平均雲量 -")+str(i)]=tmp[0][i]
    return set
def dataset4(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=data2.平均風速.loc[366+j-i]
        set[str("平均風速 -")+str(i)]=tmp[0][i]
    return set

#DataFrameに一週間の変化データを入れる

In [8]:
df= dataset1(df)#平均気温
df2= dataset2(df2)#降水量の合計
df3= dataset3(df3)#平均雲量
df4= dataset4(df4)#平均風速

In [9]:
#平均気温のデータ仕分け
X=df.drop(["平均気温"],axis=1)
y=df["平均気温"]

X_train=X[:-365]
X_test=X[-365:]
Y_train=y[:-365]
Y_test=y[-365:]

X_train.head()

平均気温 -1  平均気温 -2  平均気温 -3  平均気温 -4  平均気温 -5  平均気温 -6  平均気温 -7
datetime                                                                 
2017-01-01      6.3      6.0      4.3      6.0     11.1      6.9      6.6
2017-01-02      7.4      6.3      6.0      4.3      6.0     11.1      6.9
2017-01-03      7.2      7.4      6.3      6.0      4.3      6.0     11.1
2017-01-04      8.0      7.2      7.4      6.3      6.0      4.3      6.0
2017-01-05      8.5      8.0      7.2      7.4      6.3      6.0      4.3

In [10]:
#降水量の合計のデータ仕分け
X_2=df2.drop(["降水量の合計"],axis=1)
y_2=df2["降水量の合計"]

X_train2=X_2[:-365]
X_test2=X_2[-365:]
Y_train2=y_2[:-365]
Y_test2=y_2[-365:]

X_train2.head()

降水量の合計 -1  降水量の合計 -2  降水量の合計 -3  降水量の合計 -4  降水量の合計 -5  降水量の合計 -6  \
datetime                                                                       
2017-01-01        0.0        0.0        0.0        0.0       11.5        0.0   
2017-01-02        0.0        0.0        0.0        0.0        0.0       11.5   
2017-01-03        0.0        0.0        0.0        0.0        0.0        0.0   
2017-01-04        0.0        0.0        0.0        0.0        0.0        0.0   
2017-01-05        0.0        0.0        0.0        0.0        0.0        0.0   

            降水量の合計 -7  
datetime               
2017-01-01        0.0  
2017-01-02        0.0  
2017-01-03       11.5  
2017-01-04        0.0  
2017-01-05        0.0

In [11]:
#平均雲量のデータ仕分け
X_3=df3.drop(["平均雲量"],axis=1)
y_3=df3["平均雲量"]

X_train3=X_3[:-365]
X_test3=X_3[-365:]
Y_train3=y_3[:-365]
Y_test3=y_3[-365:]

X_train3.head()

平均雲量 -1  平均雲量 -2  平均雲量 -3  平均雲量 -4  平均雲量 -5  平均雲量 -6  平均雲量 -7
datetime                                                                 
2017-01-01      2.3      0.0      5.0      2.3      9.3      9.0      5.0
2017-01-02      2.8      2.3      0.0      5.0      2.3      9.3      9.0
2017-01-03      2.5      2.8      2.3      0.0      5.0      2.3      9.3
2017-01-04      0.5      2.5      2.8      2.3      0.0      5.0      2.3
2017-01-05      2.5      0.5      2.5      2.8      2.3      0.0      5.0

In [12]:
#平均風速のデータ仕分け
X_4=df4.drop(["平均風速"],axis=1)
y_4=df4["平均風速"]

X_train4=X_4[:-365]
X_test4=X_4[-365:]
Y_train4=y_4[:-365]
Y_test4=y_4[-365:]

X_train4.head()

平均風速 -1  平均風速 -2  平均風速 -3  平均風速 -4  平均風速 -5  平均風速 -6  平均風速 -7
datetime                                                                 
2017-01-01      1.8      3.8      2.2      4.4      3.4      1.8      1.5
2017-01-02      2.0      1.8      3.8      2.2      4.4      3.4      1.8
2017-01-03      1.4      2.0      1.8      3.8      2.2      4.4      3.4
2017-01-04      2.2      1.4      2.0      1.8      3.8      2.2      4.4
2017-01-05      2.6      2.2      1.4      2.0      1.8      3.8      2.2

In [13]:
forest = RandomForestRegressor(min_samples_leaf=3, random_state=0)
forest.fit(X_train,Y_train)#平均気温の学習
forest2 = RandomForestRegressor(min_samples_leaf=3, random_state=0)
forest2.fit(X_train2,Y_train2)#降水量の合計の学習
forest3 = RandomForestRegressor(min_samples_leaf=3, random_state=0)
forest3.fit(X_train3,Y_train3)#平均雲量の学習
forest4 = RandomForestRegressor(min_samples_leaf=3, random_state=0)
forest4.fit(X_train4,Y_train4)#平均風速の学習

RandomForestRegressor(min_samples_leaf=3, random_state=0)

In [14]:
print(forest.score(X_train,Y_train))#平均気温
print(forest2.score(X_train2,Y_train2))#降水量の合計
print(forest3.score(X_train3,Y_train3))#平均雲量
print(forest4.score(X_train4,Y_train4))#平均風速

0.9773462749416628
0.46281255783963593
0.7232747353245108
0.7012621568267491


In [15]:
#平均気温のテストデータ作成
df_pred=pd.DataFrame()
df_pred["datetime"]=pd.to_datetime(data3["年月日"])
df_pred["平均気温"]=data3["平均気温"]
#降水量の合計のテストデータ作成
df_pred2=pd.DataFrame()
df_pred2["datetime"]=pd.to_datetime(data3["年月日"])
df_pred2["降水量の合計"]=data3["降水量の合計"]
#平均雲量のテストデータ作成
df_pred3=pd.DataFrame()
df_pred3["datetime"]=pd.to_datetime(data3["年月日"])
df_pred3["平均雲量"]=data3["平均雲量"]
#平均風速のテストデータ作成
df_pred4=pd.DataFrame()
df_pred4["datetime"]=pd.to_datetime(data3["年月日"])
df_pred4["平均風速"]=data3["平均風速"]

In [16]:
def testset1(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=set.平均気温.iloc[j-i]
        set[str("平均気温 -")+str(i)]=tmp[0][i]
    return set
def testset2(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=set.降水量の合計.iloc[j-i]
        set[str("降水量の合計 -")+str(i)]=tmp[0][i]
    return set
def testset3(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=set.平均雲量.iloc[j-i]
        set[str("平均雲量 -")+str(i)]=tmp[0][i]
    return set
def testset4(set):
    tmp=np.zeros((1,8,len(set)))
    for i in range(1,8):
        for j in range(len(set)):
            tmp[0][i][j]=set.平均風速.iloc[j-i]
        set[str("平均風速 -")+str(i)]=tmp[0][i]
    return set

In [17]:
df_pred=testset1(df_pred)
df_pred2=testset2(df_pred2)
df_pred3=testset3(df_pred3)
df_pred4=testset4(df_pred4)

#各テストデータを一週間ごとの変化データにする

In [18]:
df.head()

平均気温  平均気温 -1  平均気温 -2  平均気温 -3  平均気温 -4  平均気温 -5  平均気温 -6  \
datetime                                                                 
2017-01-01   7.4      6.3      6.0      4.3      6.0     11.1      6.9   
2017-01-02   7.2      7.4      6.3      6.0      4.3      6.0     11.1   
2017-01-03   8.0      7.2      7.4      6.3      6.0      4.3      6.0   
2017-01-04   8.5      8.0      7.2      7.4      6.3      6.0      4.3   
2017-01-05   6.7      8.5      8.0      7.2      7.4      6.3      6.0   

            平均気温 -7  
datetime             
2017-01-01      6.6  
2017-01-02      6.9  
2017-01-03     11.1  
2017-01-04      6.0  
2017-01-05      4.3

In [30]:
date="20220124"#日付を指定
#平均気温
pred=pd.DataFrame()
pred["datetime"]=pd.to_datetime([date])
pred.index=pred.datetime
pred=pred.drop("datetime",axis=1)
pred["平均気温 -1"]=df_pred.iloc[-1]["平均気温"] 
#降水量の合計
pred2=pd.DataFrame()
pred2["datetime"]=pd.to_datetime([date])
pred2.index=pred2.datetime
pred2=pred2.drop("datetime",axis=1)
pred2["降水量の合計 -1"]=df_pred2.iloc[-1]["降水量の合計"] 
#平均雲量
pred3=pd.DataFrame()
pred3["datetime"]=pd.to_datetime([date])
pred3.index=pred3.datetime
pred3=pred3.drop("datetime",axis=1)
pred3["平均雲量 -1"]=df_pred3.iloc[-1]["平均雲量"] 
#平均風速
pred4=pd.DataFrame()
pred4["datetime"]=pd.to_datetime([date])
pred4.index=pred4.datetime
pred4=pred4.drop("datetime",axis=1)
pred4["平均風速 -1"]=df_pred4.iloc[-1]["平均風速"]      
                          
for i in range(2,8):
    pred["平均気温 -"+str(i)]=df_pred.iloc[-1]["平均気温 -"+str(i-1)]
    pred2["降水量の合計 -"+str(i)]=df_pred2.iloc[-1]["降水量の合計 -"+str(i-1)]
    pred3["平均雲量 -"+str(i)]=df_pred3.iloc[-1]["平均雲量 -"+str(i-1)]
    pred4["平均風速 -"+str(i)]=df_pred4.iloc[-1]["平均風速 -"+str(i-1)]

In [31]:
pred#平均気温のテストデータ

平均気温 -1  平均気温 -2  平均気温 -3  平均気温 -4  平均気温 -5  平均気温 -6  平均気温 -7
datetime                                                                 
2022-01-24      5.1      4.5      3.0      3.4      3.7      4.4      6.0

In [32]:
Y_pred =  forest.predict(pred)
Y_pred2 = forest2.predict(pred2)
Y_pred3 = forest3.predict(pred3)
Y_pred4 = forest4.predict(pred4)
print("平均気温のスコア: ",forest.score(X_train,Y_train))#平均気温
print("降水量のスコア: ",forest2.score(X_train2,Y_train2))#降水量の合計
print("平均雲量のスコア: ",forest3.score(X_train3,Y_train3))#平均雲量
print("平均風速のスコア: ",forest4.score(X_train4,Y_train4))#平均風速


print("明日の平均気温は:",round(Y_pred[0],1),"(℃)です")
print("明日の降水量の合計は:",round(Y_pred2[0],1),"(mm)です")
print("明日の平均雲量は:",round(Y_pred3[0],1),"です")
print("明日の平均風速は:",round(Y_pred4[0],1),"(m/s)です")

平均気温のスコア:  0.9773462749416628
降水量のスコア:  0.46281255783963593
平均雲量のスコア:  0.7232747353245108
平均風速のスコア:  0.7012621568267491
明日の平均気温は: 5.2 (℃)です
明日の降水量の合計は: 1.5 (mm)です
明日の平均雲量は: 6.3 です
明日の平均風速は: 2.5 (m/s)です


In [34]:

sg.theme('DarkAmber')   # デザインテーマの設定

# ウィンドウに配置するコンポーネント
layout = [  [sg.Text("電車の出発時間 時： hh"), sg.InputText()],
            [sg.Text("電車の出発時間 分： mm"), sg.InputText()],
            [sg.Button('OK'), sg.Button('キャンセル')] ]

# ウィンドウの生成
window = sg.Window('電車の時刻から支度時間と移動時間を予測', layout)

# イベントループ
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'キャンセル':
        break
    elif event == 'OK':
        if Y_pred2 > 0 and Y_pred4 >= 5:#天気の判定
            tenki = 4
        elif Y_pred2 > 0 and Y_pred4 < 5:
            tenki = 3
        elif Y_pred2 <= 0 and Y_pred4 >= 5:
            tenki = 2
        else:
            tenki = 1
    
        def test(X,y,k):#移動時間の正答率を確認
            X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
            clf=lr()
            clf.fit(X,y)
            test=np.array([[Y_pred,tenki,Y_pred4,Y_pred3,Y_pred2]])
            print("移動のスコア",clf.score(X_test,y_test))
            return clf.predict(test)
        def test2(X,y,k,idou):#準備時間の正答率を確認
            X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
            clf=lr()
            clf.fit(X,y)
            test=np.array([[Y_pred,tenki,Y_pred4,round(idou[0],0)]])
            print("準備のスコア",clf.score(X_test,y_test))
            return clf.predict(test)
        #学習データから移動時間を予測
        X_data=data1[['平均気温(℃)','天気概況','平均風速(m/s)','平均雲量(10分比)','降水量の合計(mm)']]
        Y_data=data1['移動時間(分)']

        idou=test(X_data,Y_data,tenki)
        
        #学習データから準備時間を予測
        X_data=data1[['平均気温(℃)','天気概況','平均風速(m/s)','移動時間(分)']]
        Y_data=data1['準備時間(分)']

        junbi=test2(X_data,Y_data,tenki,idou)
        #結果出力
        print("明日の移動時間は:",round(idou[0],1),"(分)です")
        print("明日の準備時間は:",round(junbi[0],1),"(分)です")
        values[0]=int(values[0])
        values[1]=int(values[1])
        if values[0] < round(idou[0],1)+round(junbi[0],1):
            values[0] = values[0] - 1
            values[1] = (60+values[1])-(round(idou[0],1)+round(junbi[0],1))
        else:
            values[1] = values[1] - (round(idou[0],1)+round(junbi[0],1))
        print("明日は",values[0],"時",int(values[1]),"分に起きてください")

window.close()

c:\users\hososan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
c:\users\hososan\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"
c:\users\hososan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
c:\users\hososan\appdata\lo

移動のスコア 0.8653170513018097
準備のスコア 0.9362396818680937
明日の移動時間は: 28.4 (分)です
明日の準備時間は: 36.4 (分)です
明日は 11 時 45 分に起きてください
